# Image Acquisition Notebook
 
--- 

This notebook contains cells pertaining to acquiring images from camera feeds over URIs. This will save all of the images locally, for further processing use in the future. As a side note, images are saved in a pre-formatted way with UIDs defined by the camera sequence number, and the date/time of the image acquision.

In [1]:
import cv2
import os

import shutil
from os import path
from time import time
from datetime import datetime

### Program Variables
Modify these variables in order to parameterize your image acquisition program.

In [2]:
# Defines how long to wait between captures
image_interval_ms = 5000

# Defines how long to capture images for. Set to 0 if
# you wish to capture images indefinitely (or until quit)
capture_elapsed_ms = 1;

# Defines the URIs to look for the streams on
camera_uris = ["192.168.1.37", "192.168.1.39"]

# Defines the extensions to pull image frames from
camera_stream_link = "/mjpg/video.mjpg"

# Defines the parent folder to save images under
save_folder = "img"

### Helper Functions
Do not modify these functions unless you intend to change how the base program functions.

In [3]:
# Gets the current time in milliseconds, from epoch
def get_current_ms():
    return int(time() * 1000) 

# Defines a save directory local to the Jupyter Notebook
def get_save_directory(cam_num):
    return f"{save_folder}/cam{cam_num}"

# Returns a unique timestamp-based name for a file
def get_unique_image_id(cam_num):
    date_stamp = f"cam{cam_num}_img_{datetime.now()}"
    date_stamp = date_stamp.replace('-', '')
    date_stamp = date_stamp.replace(':', '')
    date_stamp = date_stamp.replace('.', '')
    date_stamp = date_stamp.replace(' ', '_')
    return date_stamp + '.jpg'

# Create new URI for an image
def get_new_image_uri(cam_num):
    return get_save_directory(cam_num) + '/' + get_unique_image_id(cam_num)

# Initialize by creating the paths as needed, and storing stream links. Returns the time
# that the program finished initializing, in milliseconds from the epoch, and the links
# to each stream
def initialize():
    stream_links = {}
    
    for index, ip in enumerate(camera_uris):
        stream_links[index] = f"http://{ip}{camera_stream_link}" 

        if not path.exists(get_save_directory(index)):
            os.makedirs(get_save_directory(index))
            
    return get_current_ms(), stream_links

In [4]:
# This function is used to maintenance purposes only - it removes all files present in any 
# of the storage folders
def delete_all_images():
    response = input('This function deletes all images currently saved. ' 
                     'Are you sure you want to continue? (y/n)')
    
    if (response != 'y'):
        return
    
    print('Removing files...')    
    shutil.rmtree(save_folder)

### Acquisition Loop
Run the following cell in order to start the program. Populate the variables above depending on how you want the program to operate.

You can manually exit the program by pressing 'Q' at any time. Running this in a Jupyter Notebook may cause this feature to fail, and instead the kernel stop button should be used.

In [5]:
if __name__ == "__main__":
    print(f'Beginning time lapse recording at an interval of {image_interval_ms/1000} seconds/photo')
    print('Hold [Ctrl]+[C], or [Esc] to exit')
    start_time_ms, stream_links = initialize()
    
    while True if capture_elapsed_ms <= 0 else ((get_current_ms() - start_time_ms) > capture_elapsed_ms):
        for camera_id in stream_links.keys():
            stream = cv2.VideoCapture(stream_links[camera_id])
            ret, frame = stream.read()

            cv2.imwrite(get_new_image_uri(camera_id), frame) 
            
        if cv2.waitKey(image_interval_ms) == 27:
            sys.exit()

Beginning time lapse recording at an interval of 5.0 seconds/photo
Hold [Ctrl]+[C], or [Esc] to exit
